# RAG 实现

## 加载文档

### 从文件读取，并实现拆分

In [1]:
import os
os.getcwd()

'/Users/xuehongwei/github/illufly/notes/core'

In [4]:
from illufly.importer import MarkdownFileImporter

t = MarkdownFileImporter(dir="./", chunk_size=550)
t(verbose=True)
t.documents

[Document(text='# illufly 设计理念介绍...', metadata='{"source": "./docs/设计理念.md"}'),
 Document(text='一般的开发框架们为了开发者使用，会提供自己的类定义，例如用下面的代码来替代：...', metadata='{"source": "./docs/设计理念.md"}'),
 Document(text='现在，要求你准确记住这些类的名字以及该从哪里引用，就开始慢慢形成挑战了。...', metadata='{"source": "./docs/设计理念.md"}'),
 Document(text=''role': 'user',...', metadata='{"source": "./docs/设计理念.md"}'),
 Document(text='# 常见的智能体推理模式和 illufly 的实现...', metadata='{"source": "./docs/推理模式.md"}'),
 Document(text='REWOO的全称是Reason without Observation，旨在通过以下方式改进ReACT风格的Agent架构：...', metadata='{"source": "./docs/推理模式.md"}'),
 Document(text='- 重规划器Replanner：根据执行情况和反馈调整计划。...', metadata='{"source": "./docs/推理模式.md"}'),
 Document(text='Self-Discover由Google研究人员提出，允许大型语言模型在没有明确标签情况下，自主选择并组合原子推理模块，生成推理结构。包含两个阶段：...', metadata='{"source": "./docs/推理模式.md"}'),
 Document(text='- **自我迭代的改进**：自动收集智能体的反思能力和规划能力的评测依据和微调依据，确保智能体可以不断优化和提升。...', metadata='{"source": "./docs/推理模式.md"}')]

In [3]:
print(t.documents[1].text)

一般的开发框架们为了开发者使用，会提供自己的类定义，例如用下面的代码来替代：

```python
[
    SystemMessage('你是一个AI助手'),
    UserMessage('你好'),
    AIMessage('有什么可以帮你')
]
```

然后要求开发者尽量使用已经创建的 XXMessage 类来封装所有关于消息格式的功能。
到目前为止，这看起来很不错，也完全符合一般的设计原则。

但问题会逐渐显现。

首先是类定义的体系，仅仅上面几个类是不够的。例如：

- 你一定需要基类，比如：`BaseMessage`
- 也许你需要区分出工具消息，比如： `ToolMessage`？但这样也许不太够，因为要区分大模型返回的和工具执行的，也许是这样：`ToolCallMessage` 和 `ToolRespMessage`
- 如果你要区分携带部分信息的消息，可能还要增加 `UserMessageChunk`、`AIMessageChunk`，以及`ToolCallMessageChunk` 和 `ToolRespMessageChunk`
- 多模态的能力中图片的消息该如何定义？是否要增加 `ImageMessage`、`AudioMessage`、`VideoMessage`的定义？以及对应的 `ImageMessageChunk`、`AudioMessageChunk`、`VideoMessageChunk`？

有了这些类，你要先记住才能开始做其他的，例如：
```python
from xxxxxx.xxxxxx.messages import UserMessage, AIMessage, ImageMessage, VideoMessage ...
```

现在，要求你准确记住这些类的名字以及该从哪里引用，就开始慢慢形成挑战了。

...

类似的事情会在很多地方发生。

你的初衷是，通过这些新类的定义来简化工作，但是现在不得不记住很多新东西。

### illufly 中的解决方案

```python
[
    {
        'role': 'system',
        'content': '你是一个AI助手'
    },
    {


## 向量数据库

### 预先从 Embeddings 构建文档

In [1]:
from illufly.embeddings import TextEmbeddings
from illufly.importer import MarkdownFileImporter
from illufly.retriever import FaissDB

# 导入文档
local = MarkdownFileImporter(dir="./", chunk_size=550)
local(verbose=True)

# 文档嵌入
emb = TextEmbeddings()
emb(local.documents, verbose=True)

# 文档入库
faiss = FaissDB(emb, train=False)

# 查询
faiss("智能体有哪些类型？", verbose=True)

  0s [INFO] [0.706] ./docs/推理模式.md: - **自我迭代的改进**：自动收集智能体的反思能力和规划能力的评测依据和微调依据，确保智能体可以不断优化和提升。...
  0s [INFO] [0.867] ./docs/推理模式.md: # 常见的智能体推理模式和 illufly 的实现...
  0s [INFO] [0.934] ./docs/设计理念.md: 'role': 'user',...
  0s [INFO] [0.951] ./docs/推理模式.md: Self-Discover由Google研究人员提出，允许大型语言模型在没有明确标签情况下，自主选择并组合原子推理模块，生成推理结构。包含两个阶段：...
  0s [INFO] [0.987] ./docs/设计理念.md: # illufly 设计理念介绍...


[(0.70617646,
  Document(text='- **自我迭代的改进**：自动收集智能体的反思能力和规划能力的评测依据和微调依据，确保智能体可以不断优化和提升。...', metadata='{"source": "./docs/推理模式.md", "embeddings": [-0.06955220550298691, -0.040011003613471985, -0.03216838...')),
 (0.86662793,
  Document(text='# 常见的智能体推理模式和 illufly 的实现...', metadata='{"source": "./docs/推理模式.md", "embeddings": [-0.03823075070977211, 0.025731494650244713, 0.0117180533...')),
 (0.9336294,
  Document(text=''role': 'user',...', metadata='{"source": "./docs/设计理念.md", "embeddings": [-0.061874520033597946, 0.01978103630244732, -0.015418602...')),
 (0.9512944,
  Document(text='Self-Discover由Google研究人员提出，允许大型语言模型在没有明确标签情况下，自主选择并组合原子推理模块，生成推理结构。包含两个阶段：...', metadata='{"source": "./docs/推理模式.md", "embeddings": [-0.04360850527882576, -0.003100050613284111, -0.01595692...')),
 (0.98729813,
  Document(text='# illufly 设计理念介绍...', metadata='{"source": "./docs/设计理念.md", "embeddings": [-0.09724362939596176, 0.012529165484011173, -0.007420137...'))]

### Importer + Embeddings + VectorDB 

In [2]:
from illufly.embeddings import TextEmbeddings
from illufly.vectordb import FaissDB

# 文档嵌入
db = FaissDB(TextEmbeddings())
db.add(["普鸿是一家做数字消防业务的公司"])
db.add(["幻蝶是一家AI技术公司"])

# 查询
db("普鸿是啥？", top_k=5, verbose=True)

  0s [INFO] [0.487] unknown: 普鸿是一家做数字消防业务的公司
  0s [INFO] [1.348] unknown: 幻蝶是一家AI技术公司


[(0.48679453,
  Document(text='普鸿是一家做数字消防业务的公司', metadata='{"source": "unknown", "embeddings": [-0.09413302689790726, 0.01354235503822565, -0.03795371949672699...')),
 (1.3482788,
  Document(text='幻蝶是一家AI技术公司', metadata='{"source": "unknown", "embeddings": [-0.08771800994873047, 0.011481866240501404, 0.00165668292902410...'))]

In [3]:
from illufly.embeddings import TextEmbeddings
from illufly.vectordb import FaissDB

# 文档嵌入
db = FaissDB(TextEmbeddings())
db.add_files("./", chunk_size=550)
# 查询
db("智能体是啥？", top_k=5, verbose=True)

  0s [INFO] [0.680] ./docs/推理模式.md: - **自我迭代的改进**：自动收集智能体的反思能力和规划能力的评测依据和微调依据，确保智能体可以不断优化和提升。...
  0s [INFO] [0.953] ./docs/推理模式.md: # 常见的智能体推理模式和 illufly 的实现...
  0s [INFO] [0.960] ./docs/设计理念.md: 'role': 'user',...
  0s [INFO] [1.031] ./docs/推理模式.md: Self-Discover由Google研究人员提出，允许大型语言模型在没有明确标签情况下，自主选择并组合原子推理模块，生成推理结构。包含两个阶段：...
  0s [INFO] [1.038] ./docs/设计理念.md: # illufly 设计理念介绍...


[(0.67994213,
  Document(text='- **自我迭代的改进**：自动收集智能体的反思能力和规划能力的评测依据和微调依据，确保智能体可以不断优化和提升。...', metadata='{"source": "./docs/推理模式.md", "embeddings": [-0.06955220550298691, -0.040011003613471985, -0.03216838...')),
 (0.95295113,
  Document(text='# 常见的智能体推理模式和 illufly 的实现...', metadata='{"source": "./docs/推理模式.md", "embeddings": [-0.03823075070977211, 0.025731494650244713, 0.0117180533...')),
 (0.9598516,
  Document(text=''role': 'user',...', metadata='{"source": "./docs/设计理念.md", "embeddings": [-0.061874520033597946, 0.01978103630244732, -0.015418602...')),
 (1.0309153,
  Document(text='Self-Discover由Google研究人员提出，允许大型语言模型在没有明确标签情况下，自主选择并组合原子推理模块，生成推理结构。包含两个阶段：...', metadata='{"source": "./docs/推理模式.md", "embeddings": [-0.04360850527882576, -0.003100050613284111, -0.01595692...')),
 (1.0380639,
  Document(text='# illufly 设计理念介绍...', metadata='{"source": "./docs/设计理念.md", "embeddings": [-0.09724362939596176, 0.012529165484011173, -0.007420137...'))]

## 结合 ChatAgent 构建 RAG

In [4]:
# 创建一个工具
from illufly.types import BaseAgent, Template, EventBlock, Messages
from illufly.embeddings import TextEmbeddings
from illufly.rag import FaissDB
from illufly.chat import ChatQwen

# 文档嵌入
faiss_db = FaissDB(TextEmbeddings(), top_k=1)
faiss_db.load("./", chunk_size=500)

## 声明大模型
qwen = ChatQwen(
    knowledge=["illufly实现了多智能体结构", faiss_db],
    memory=[("system", "你是一个作家")]
)

Messages("ReAct是什么意思", style="text").messages[-1].content

qwen("ReAct是什么意思？", verbose=True)
qwen.memory

[INFO] 记住 10 轮对话
ReAct是一种结合了推理（reasoning）和行动（acting）的语言模型设计模式，该模式被设计用来增强语言模型在处理多样化的语言推理和决策任务时的能力。ReAct模式的核心在于它模仿了人类思考和行动的过程，通过一个循环流程来进行问题解决，这个流程包括了思考（Thought）、行动（Action）和观察（Observation）三个步骤。具体来说：

1. **思考（Thought）** - 在这个阶段，智能体会基于当前的信息进行思考，形成一个行动计划。
2. **行动（Action）** - 根据形成的计划，智能体会采取具体的行动，这可能涉及与外部环境的交互。
3. **观察（Observation）** - 智能体随后会观察其行动的结果，并根据新的信息更新其内部状态。

通过这种循环的方式，ReAct使得智能体能够逐步地解决问题，每一步都会基于之前的经验和新获得的信息进行调整。这种方法的一个优点是它让决策过程变得更加透明，更容易被人类理解和检查。

然而，ReAct模式也存在一些限制，比如由于依赖于大型语言模型（LLMs），它可能会导致内容生成的不稳定性，以及处理复杂任务时的成本和时间开销问题。此外，由于每次交互都需要时间来获取LLM的响应，这也影响了系统的响应速度和用户体验。
[USAGE] {"input_tokens": 380, "output_tokens": 281, "total_tokens": 661}


[{'role': 'system', 'content': '你是一个作家'},
 {'role': 'user',
  'content': '已知知识：\nillufly实现了多智能体结构已知知识：\n# 常见的智能体推理模式和 illufly 的实现\n\n## 常见推理模式\n\nReAct模式是最早出现的Agent设计模式，目前也是应用最广泛的。从ReAct出发，有两条发展路线：一条更偏重Agent的规划能力，包括REWOO、Plan & Execute、LLM Compiler；另一条更偏重反思能力，包括Basic Reflection、Reflexion、Self Discover、LATS。\n\nillufly中将实现所有这些推理模式。\n\n### ReAct\n\nReAct的概念来自论文《ReAct: Synergizing Reasoning and Acting in Language Models》，提出了一种结合语言模型中的推理（reasoning）和行动（acting）来解决多样化语言推理和决策任务的方法。ReAct提供了一种更易于人类理解、诊断和控制的决策和推理过程。\n\nReAct的核心思想是模拟人类思考和行动的过程，通过Thought、Action、Observation的循环，一步步解决目标问题。\n\nReAct模式存在以下不足：\n\n- LLM大模型的通病，即产出内容不稳定，对复杂问题的分析和解决存在波动。\n- 成本高，无法控制输入内容，复杂任务可能导致Token过量消耗。\n- 响应时间长，LLM响应时间是秒级以上，且在ReAct模式下更加不可控，需要采用异步方式，影响用户体验和应用场景选择。\n\n### REWOO\n\nREWOO的全称是Reason without Observation，旨在通过以下方式改进ReACT风格的Agent架构：\n\n- 生成一次性使用的完整工具链，减少token消耗和执行时间。'},
 {'role': 'assistant', 'content': 'OK'},
 {'role': 'user', 'content': 'ReAct是什么意思？'},
 {'role': 'assistant',
  'content': 'ReAct是一种结合了推理（reasoning

In [2]:
faiss_db("ReAct是什么意思")

[Document(text="# 常见的智能体推理模式和 illufly 的实现...", meta=['source', 'id', 'embeddings', 'distance'])]

In [3]:
qwen.get_knowledge("ReAct是什么意思")

['# 常见的智能体推理模式和 illufly 的实现\n\n## 常见推理模式\n\nReAct模式是最早出现的Agent设计模式，目前也是应用最广泛的。从ReAct出发，有两条发展路线：一条更偏重Agent的规划能力，包括REWOO、Plan & Execute、LLM Compiler；另一条更偏重反思能力，包括Basic Reflection、Reflexion、Self Discover、LATS。\n\nillufly中将实现所有这些推理模式。\n\n### ReAct\n\nReAct的概念来自论文《ReAct: Synergizing Reasoning and Acting in Language Models》，提出了一种结合语言模型中的推理（reasoning）和行动（acting）来解决多样化语言推理和决策任务的方法。ReAct提供了一种更易于人类理解、诊断和控制的决策和推理过程。\n\nReAct的核心思想是模拟人类思考和行动的过程，通过Thought、Action、Observation的循环，一步步解决目标问题。\n\nReAct模式存在以下不足：\n\n- LLM大模型的通病，即产出内容不稳定，对复杂问题的分析和解决存在波动。\n- 成本高，无法控制输入内容，复杂任务可能导致Token过量消耗。\n- 响应时间长，LLM响应时间是秒级以上，且在ReAct模式下更加不可控，需要采用异步方式，影响用户体验和应用场景选择。\n\n### REWOO\n\nREWOO的全称是Reason without Observation，旨在通过以下方式改进ReACT风格的Agent架构：\n\n- 生成一次性使用的完整工具链，减少token消耗和执行时间。',
 'illufly实现了多智能体结构']

In [6]:
qwen.knowledge

{<illufly.community.faiss.faiss_cpu.FaissDB at 0x1223d4760>,
 Document(text='illufly实现了多智能体结构', metadata='{"source": "unknown"}')}

## ReRank

In [3]:
import dashscope
from http import HTTPStatus

def text_rerank(): 
    resp = dashscope.TextReRank.call(
        model=dashscope.TextReRank.Models.gte_rerank,
        query="什么是文本排序模型",
        documents=[
            "文本排序模型广泛用于搜索引擎和推荐系统中，它们根据文本相关性对候选文本进行排序",
            "量子计算是计算科学的一个前沿领域",
            "预训练语言模型的发展给文本排序模型带来了新的进展"
        ],
        top_n=10,
        return_documents=True
    )
    if resp.status_code == HTTPStatus.OK:
        print(resp)
    else:
        print(resp)

text_rerank() 

{"status_code": 200, "request_id": "23b62943-8321-9a17-aa03-21a9ff13d36a", "code": "", "message": "", "output": {"results": [{"index": 0, "relevance_score": 0.7314485774089865, "document": {"text": "文本排序模型广泛用于搜索引擎和推荐系统中，它们根据文本相关性对候选文本进行排序"}}, {"index": 2, "relevance_score": 0.5831720487049298, "document": {"text": "预训练语言模型的发展给文本排序模型带来了新的进展"}}, {"index": 1, "relevance_score": 0.04973238644524712, "document": {"text": "量子计算是计算科学的一个前沿领域"}}]}, "usage": {"total_tokens": 79}}


In [6]:
from illufly.types import Document
from illufly.embeddings import TextEmbeddings

embeddings = TextEmbeddings()
docs = [Document("这是一个测试文本", meta={"source": "test"})]
embeddings(docs)
print(embeddings.last_output[0].meta['embeddings'])

[-0.10436907410621643, 0.043999478220939636, -0.07047602534294128, -0.026495756581425667, -0.03247123211622238, -0.04715053364634514, 0.016571857035160065, 0.10183285921812057, -0.016648711636662483, 0.017609398812055588, -0.01869497448205948, 0.015697630122303963, -0.00154790747910738, -0.01839716173708439, 0.008958409540355206, -0.030261650681495667, 0.03414282575249672, -0.03539172187447548, -0.0143910963088274, -0.05752595514059067, -0.08669242262840271, 0.005749713629484177, 0.024574382230639458, 0.029358604922890663, 0.024324603378772736, 0.07962176203727722, -0.04718896001577377, -0.05495131388306618, 0.0182146318256855, -0.016495000571012497, 0.0754331722855568, 0.009597266092896461, 0.051992397755384445, -0.07232054322957993, -0.014506378211081028, -0.007320437580347061, -0.0015947410138323903, -0.0529530867934227, -0.0607154406607151, 0.009943113662302494, 0.05560458078980446, -0.03587206453084946, -0.003143849316984415, -0.03737073764204979, -0.004952343180775642, -0.0400798

## 构造问题

In [4]:
import json
qa = json.loads(qwen.last_output)
qa

{'question': '如何在家制作美味的披萨？',
 'more_questions': ['在家做披萨需要哪些材料？',
  '披萨饼皮怎么做才更酥脆？',
  '如何选择合适的披萨酱？',
  '应该在披萨上加什么配料？',
  '如何控制烤箱温度来烤披萨？',
  '有没有简单易学的披萨食谱？',
  '怎样可以让披萨更香？',
  '披萨烤多久才熟？',
  '如何做出边缘金黄的披萨？',
  '做披萨时面团如何发酵？',
  '如何防止披萨底部变湿？',
  '披萨上的奶酪应该什么时候加？',
  '如何判断披萨是否烤好？',
  '披萨出炉后应该如何切片？',
  '如何保存剩余的披萨？',
  '如何让披萨口感更好？',
  '披萨做好后如何保持热度？',
  '如何制作不同口味的披萨？',
  '如何避免披萨太油腻？',
  '如何在家制作健康版披萨？'],
 'qa': [{'Q': '如何在家制作美味的披萨？',
   'A': '首先准备好所需材料，如面粉、酵母、番茄酱等，然后按照食谱制作饼皮并添加喜欢的配料，最后放入预热好的烤箱中烘烤至金黄色即可。'},
  {'Q': '如何在家制作美味的披萨？', 'A': '可以先上网查找一些简单的披萨食谱，根据步骤准备材料和烤制。'},
  {'Q': '如何在家做蛋糕？', 'A': '蛋糕的做法与披萨完全不同，需要准备不同的原料和工具。'},
  {'Q': '如何选择合适的披萨酱？', 'A': '根据个人口味选择，比如番茄酱、白酱或BBQ酱等。'},
  {'Q': '如何选择合适的披萨酱？', 'A': '可以根据配料来搭配，比如海鲜配白酱，肉类配番茄酱。'},
  {'Q': '如何训练一只狗坐下？', 'A': '用奖励的方法，每次它坐下就给点心。'},
  {'Q': '如何让披萨更香？', 'A': '可以在披萨上撒一些罗勒叶或者牛至增加香气。'},
  {'Q': '如何让披萨更香？', 'A': '加入适量的香料如迷迭香或百里香可以提升香味。'},
  {'Q': '如何种植西红柿？', 'A': '挑选健康的种子，播种后定期浇水施肥。'},
  {'Q': '如何控制烤箱温度来烤披萨？', 'A': '通常设置在220度左右，具体根据烤箱类型调整。'},
  {'Q': 

In [2]:
import uuid
uuid.uuid4()

UUID('f80256e5-604a-47bc-98c0-7ba4186a986e')

In [1]:
from illufly.chat import ChatQwen
from illufly.types import Template

qwen = ChatQwen(
    memory=[Template("RAG/QUESTION"), "请开始"],
    template_binding={"count": lambda x: 5}
)
qwen("任意")

```markdown
@metadata tag='related'
**Question:**
如何在家里制作美味的比萨？

**Answer:**
首先准备面团，然后加入番茄酱，放上你喜欢的配料，最后撒上奶酪，放入预热至200度的烤箱中烤约15分钟即可。

@metadata tag='related'
**Question:**
有没有简单的方法在家做比萨？

**Answer:**
可以买现成的比萨饼底，加上自己喜欢的食材，放入烤箱烤制即可。

@metadata tag='related'
**Question:**
自己动手做比萨需要注意哪些步骤？

**Answer:**
注意面团发酵时间，酱料的选择，配料的新鲜程度以及烘烤温度和时间。

@metadata tag='related'
**Question:**
在家中怎样才能做出餐厅级别的比萨？

**Answer:**
选用高质量的原材料，掌握正确的烘焙技巧，尝试不同的配料组合。

@metadata tag='related'
**Question:**
想在家里给家人做一顿特别的晚餐，有什么推荐？

**Answer:**
自制比萨是个不错的选择，可以根据家人的口味定制各种风味。

@metadata tag='unrelated'
**Question:**
如何训练一只小狗成为导盲犬？

**Answer:**
从小狗三个月大时开始基础训练，之后进行专业技能训练，通过测试后与视障者配合训练。

@metadata tag='unrelated'
**Question:**
怎样才能提高自己的编程能力？

**Answer:**
多练习，参与开源项目，阅读优秀的代码，不断学习新的技术和框架。

@metadata tag='unrelated'
**Question:**
为什么我的盆栽植物总是长不好？

**Answer:**
可能是浇水过多或过少，光照不足，或者土壤不适合植物生长，需要调整养护方法。

@metadata tag='unrelated'
**Question:**
如何挑选一双适合跑步的运动鞋？

**Answer:**
要选择适合自己脚型的鞋子，注意鞋子的缓震性和透气性，最好在下午试穿以确保大小合适。

@metadata tag='unrelated'
**Ques

"@metadata tag='related'\n**Question:**\n如何在家里制作美味的比萨？\n\n**Answer:**\n首先准备面团，然后加入番茄酱，放上你喜欢的配料，最后撒上奶酪，放入预热至200度的烤箱中烤约15分钟即可。\n\n@metadata tag='related'\n**Question:**\n有没有简单的方法在家做比萨？\n\n**Answer:**\n可以买现成的比萨饼底，加上自己喜欢的食材，放入烤箱烤制即可。\n\n@metadata tag='related'\n**Question:**\n自己动手做比萨需要注意哪些步骤？\n\n**Answer:**\n注意面团发酵时间，酱料的选择，配料的新鲜程度以及烘烤温度和时间。\n\n@metadata tag='related'\n**Question:**\n在家中怎样才能做出餐厅级别的比萨？\n\n**Answer:**\n选用高质量的原材料，掌握正确的烘焙技巧，尝试不同的配料组合。\n\n@metadata tag='related'\n**Question:**\n想在家里给家人做一顿特别的晚餐，有什么推荐？\n\n**Answer:**\n自制比萨是个不错的选择，可以根据家人的口味定制各种风味。\n\n@metadata tag='unrelated'\n**Question:**\n如何训练一只小狗成为导盲犬？\n\n**Answer:**\n从小狗三个月大时开始基础训练，之后进行专业技能训练，通过测试后与视障者配合训练。\n\n@metadata tag='unrelated'\n**Question:**\n怎样才能提高自己的编程能力？\n\n**Answer:**\n多练习，参与开源项目，阅读优秀的代码，不断学习新的技术和框架。\n\n@metadata tag='unrelated'\n**Question:**\n为什么我的盆栽植物总是长不好？\n\n**Answer:**\n可能是浇水过多或过少，光照不足，或者土壤不适合植物生长，需要调整养护方法。\n\n@metadata tag='unrelated'\n**Question:**\n如何挑选一双适合跑步的运动鞋？\n\n**Answer:**\n要选择适合自己脚型的鞋子，注意鞋子的缓震性和透气性